<a href="https://colab.research.google.com/github/zinojeng/diabetes/blob/main/glycemic_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Glycemic Index Recommendation using Google's Gemini Pro Vision | Python | Google AI Studio

[**Link to my YouTube Channel**](https://www.youtube.com/BhaveshBhatt8791?sub_confirmation=1)

Click on the link below to open a Colab version of the notebook. You will be able to create your own version.

<a href="https://colab.research.google.com/github/bhattbhavesh91/glycemic-index-google-gemini/blob/main/glycemic-index-notebook.ipynb" target="_blank"><img height="40" alt="Run your own notebook in Colab" src = "https://colab.research.google.com/assets/colab-badge.svg"></a>

# Installation

In [ ]:
!pip install -q google-generativeai==0.3.1
!pip install -q gradio

# Imports

In [2]:
import google.generativeai as genai
from pathlib import Path
import gradio as gr

# Version

In [3]:
genai.__version__

'0.3.1'

验证下是否安装正确

In [5]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-pro
models/gemini-pro-vision


# Secret Key

In [4]:
from google.colab import userdata

genai.configure(api_key = userdata.get('GEMINI_KEY'))

# Configurations

In [6]:
# Set up the model
generation_config = {
  "temperature": 0.4,
  "top_p": 1,
  "top_k": 32,
  "max_output_tokens": 4096,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  }
]

# Model Instance

In [7]:
model = genai.GenerativeModel(model_name = "gemini-pro-vision",
                              generation_config = generation_config,
                              safety_settings = safety_settings)

# Image Input Function

In [8]:
def input_image_setup(file_loc):
    if not (img := Path(file_loc)).exists():
        raise FileNotFoundError(f"Could not find image: {img}")

    image_parts = [
        {
            "mime_type": "image/jpeg",
            "data": Path(file_loc).read_bytes()
            }
        ]
    return image_parts

# Response Function

In [9]:
def generate_gemini_response(input_prompt, image_loc):

    image_prompt = input_image_setup(image_loc)
    prompt_parts = [input_prompt, image_prompt[0]]
    response = model.generate_content(prompt_parts)
    return response.text

# Prompt

In [12]:
input_prompt = """
As an expert specializing in assessing the suitability of fruits and foods for individuals with diabetes, your task involves analyzing input images featuring various food items. Your first objective is to identify the type of fruit or food present in the image. Subsequently, you must determine the glycemic index and the estimation of calories of the identified item. Based on this glycemic index and calories, provide recommendations on whether individuals with diabetes can include the detected food in their diet. If the food is deemed suitable, specify the recommended quantity for consumption. response all in zh-tw.
               """

# Defining the Interface

In [ ]:
def upload_file(files):
    file_paths = [file.name for file in files]
    if file_paths:
        response = generate_gemini_response(input_prompt, file_paths[0])
    return  file_paths[0], response

with gr.Blocks() as demo:
    gr.Markdown("## 糖尿病飲食建議 by Gemini-pro-vision. ")
    gr.Markdown("coded by Dr. Tseng, Tungs' Taichung Metroharbor Hospital.")
    file_output = gr.Textbox()
    image_output = gr.Image()
    combined_output = [image_output, file_output]
    upload_button = gr.UploadButton("點擊上傳您的食物圖片",
                                    file_types=["image"],
                                    file_count="multiple")
    upload_button.upload(upload_file, upload_button, combined_output)

demo.launch(debug=True, server_port=9001,share=True, inbrowser=True)

In [ ]:
!pip install streamlit

In [23]:
import streamlit as st
from PIL import Image
import io

# 假設 generate_gemini_response 是您先前定義的函數
def generate_gemini_response(input_prompt, file_path):
    # 這裡添加您的 generate_gemini_response 函數的內容
    return "Response based on the image"

def upload_file(uploaded_file):
    if uploaded_file is not None:
        # To read file as bytes:
        bytes_data = uploaded_file.getvalue()
        image = Image.open(io.BytesIO(bytes_data))
        response = generate_gemini_response("Your input prompt", uploaded_file.name)
        return uploaded_file.name, image, response
    return None, None, None

# Streamlit 應用界面
def main():
    st.title("糖尿病飲食建議 by Gemini-pro-vision.")
    st.markdown("coded by Dr. Tseng, Tungs' Taichung Metroharbor Hospital.")

    uploaded_file = st.file_uploader("點擊上傳您的食物圖片", type=["png", "jpg", "jpeg"])
    file_name, image, response = upload_file(uploaded_file)

    if file_name is not None:
        st.image(image, caption=file_name)
        st.text(response)

# 執行 Streamlit 應用
if __name__ == "__main__":
    main()


2024-01-04 19:42:23.433 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
